In [ ]:
'''
TAB LAYOUT IDEA:
TAB 1: Intra-Groupchats
TAB 2: Inter-Groupchats
TAB 3: Intra-pm's
TAB 4: Inter-pm's
TAB 5: All about me
TAB 6: Settings?
TAB 7: 

'''

In [1]:
from getData import df
import pandas as pd

In [4]:
from scipy import interpolate
from bokeh.plotting import figure

In [ ]:
authors = df.Author.unique()
pm_df = df[df.Conversation.isin(authors)]
authors = list(df.Author.unique())
authors.remove('Miles Keating')

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)



In [ ]:
category = 'Running Word Count'
vals = pm_df.loc[pm_df.Author == 'Bilo']

vals = vals[['Datetime', category]]
vals.set_index('Datetime',inplace=True)

temp = vals.copy()

vals= vals.resample('7D').first()
vals.fillna(value=0,inplace=True)

vals = vals.rolling(window=4).mean()

vals['difference'] = vals[~vals['Running Word Count'].isnull()].diff()


vals.fillna(value=0,inplace=True)

vals = vals.rolling(window=4).mean()

vals.fillna(value=0,inplace=True)

from scipy.signal import savgol_filter
vals.fillna(value=0,inplace=True)

vals[category] = savgol_filter(vals[category], 11, 3)
vals.fillna(value=0,inplace=True)

# Each author get's their own CDS

In [ ]:
cds_num = []
conversations = df.Conversation.unique()
for conversation in conversations:
    cds_num.append(len(df[df.Conversation == conversation].Author.unique()))
    
cds_num = max(cds_num)
cds_num

In [ ]:
CDSlist = []
fig = figure(x_axis_type='datetime')
reset_dict = {x:[] for x in df.columns.unique()}
debug = pd.DataFrame(reset_dict)

    
subset = df.copy()
#subset.set_index('Datetime', inplace=True)
subset = subset[subset.Conversation == 'Plasma Lab']


for i, author in enumerate(subset.Author.unique()):
    CDSlist.append(ColumnDataSource(reset_dict))
    fig.line(source=CDSlist[i],
            x='Datetime', y = 'Sent Words', legend_label='none')

    
CDSlist[-1].data.update(subset[subset.Author == 'Miles Keating'])

def make_dataset(local_df):
    for i, author in enumerate(local_df.Author.unique()):
        subset = local_df[local_df.Author == author]
        new_src = ColumnDataSource(subset)
        CDSlist[i].data.update(new_src.data) #update the data of the source
        
show(fig)

In [ ]:
from bokeh.palettes import turbo, Category20_20

colors = Category20_20
len(colors)

In [2]:
'''
cds_label
CDSlist
group_df
metric_seleection
conversations
authors
height
width
'''

from getData import df
from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, Legend, LegendItem
from bokeh.models.widgets import RadioButtonGroup, Select, CheckboxGroup
from bokeh.layouts import column, row
from bokeh.palettes import turbo, Category20_20, Category10_10

output_notebook()

#Organize data


def tab1(doc):
    
    reset_data = {x:[] for x in df.columns.unique()}
    global colors
    colors = Category20_20
    
    #Create Figures
    height = 300
    width = 1000
    
    top = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    legend = Legend(items=[])
    top.add_layout(legend, 'right')
    bot = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    
    group_df = df.drop(df[df.Conversation.isin(list(df.Author.unique()))].index)

    all_authors = group_df.Author.unique()
    conversations = list(group_df.Conversation.unique())
    global old_conversation
    old_conversation = [conversations[0]]
    
    legend_items =[]



    def update_author(attr, old, new):
        subset = group_df[group_df.Conversation == conversation_select.value]
        
        if len(old) > len(new): #signifies unchecking a box
            change = checkbox.labels[list(set(old) - set(new))[0]]
            new_data = reset_data
        else: #signifies checking a box
            index = list(set(new) - set(old))[0]
            change = checkbox.labels[list(set(new) - set(old))[0]]
            new_data = subset[subset.Author == change]
            renders[change].glyph.line_color=colors[index]
        items=[]
        for x in checkbox.active:
            x= checkbox.labels[x]
            items.append((x,[renders[x]]))
        top.legend.items=items

        src[change].data.update(new_data)
        
    def update_conversation(attr, old, new):
        subset = group_df[group_df.Conversation == new]
        
        for author in all_authors:
            src[author].data.update(reset_data)

            
        checkbox.active = []
        checkbox.labels = list(subset.Author.unique())
        
        global colors
        if len(subset.Author.unique()) > 20:
            colors = turbo(len(subset.Author.unique()))
        elif len(subset.Author.unique()) > 10:
            colors = Category20_20
        else:
            colors = Category10_10

        
        
        
    def update_column(attr, old, new):
        newval = column_radio.labels[new]
        for r in renders:
            renders[r].glyph.y = newval
            
    #Create Widgets
    column_selection = ['Sent Messages', 'Sent Words','WPM']
    column_radio = RadioButtonGroup(labels= column_selection,
                                   active=1)
    conversation_select = Select(title='Conversation',
                                value =conversations[0],
                                options=conversations)
    

    subset = group_df[group_df.Conversation == conversation_select.value]
    authors = list(subset.Author.unique())

    # Create Checkbox Widget

    checkbox = CheckboxGroup(labels=authors, active=[])
    
    #Populate the src and tie each one to a render
    src = {} #Dictionary of CDS' where each author maps to a CDS
    renders = {} # Dictionary of renderers where each author maps to a renderer
    for author in all_authors:
        src[author] = ColumnDataSource(reset_data)
        renders[author] = top.line(source=src[author],
                           x='Datetime', y = column_radio.labels[column_radio.active])
        legend_items.append([(author,[renders[author]])])
        
    
    top.legend.items = []

    checkbox.on_change('active', update_author)
    conversation_select.on_change('value', update_conversation)
    column_radio.on_change('active', update_column)
    
    layout = column(column_radio,
                    row(column(conversation_select, checkbox), top))
    doc.add_root(layout)


    
show(tab1)

print('done')

Loading BokehJS ...

done


In [ ]:
from bokeh.models import Button

def test(doc):
    fig = figure()
    y = [12, 14, 16, 3, 1, 5, 3]

    x = [1, 2, 3, 4, 5, 6, 7]

    button = Button(label = 'remove', button_type='success')
    debug = []
    for q in [10, 14, 17, 22, 15, 32, 41, 56, 73, 121]:
            y2 = [i*q for i in y]
            debug.append(fig.circle(x=x, y=y2, legend= str(q)))
    def update():
        for key in debug:
            key.visible = not key.visible


    r1 = fig.circle(x=x,y=y)
    

    button.on_click(update)

    layout = row(button,fig)
    doc.add_root(layout)
show(test)

# Inter-PM Tab

In [ ]:
'''
Inter-PM means the checkbox group will filter by Conversations instead of Authors. The analysis will compare
all conversations with each other, will it follow the exact same format?? maybe it will. idk

I'm going to try this tab in a different way where I create a new CDS for every non-checkbox callback,
this way the plot doesn't get overloaded with glyphs
'''

from bokeh.io import show, output_notebook
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, IndexFilter, HoverTool, Panel
from bokeh.models.widgets import RadioButtonGroup, CheckboxGroup
from bokeh.layouts import column, row
from bokeh.palettes import turbo


# Import the data
from getData import df

pm_df = df.drop(df[~df.Conversation.isin(list(df.Author.unique()))].index)

metric_selection = ['Sent Messages', 'Sent Words','WPM']
conversations = list(group_df.Conversation.unique())
authors = []

height = 300
width = 1000

def tab3(doc):
    
    #Create Figures
    
    top = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    bot = figure(x_axis_type='datetime', frame_height=height, frame_width=width)

    #Create Widgets

    metric_radio = RadioButtonGroup(labels= metric_selection,
                                   active=1)
    #Checkbox group to select which conversation glyph to overlay
    checkbox = CheckboxGroup(labels=authors, active=[])
    
    
    conversation_select = Select(title='Conversation',
                                value =conversations[0],
                                options=conversations)
    
    subset = pm_df[pm_df.Conversation == conversations[0]]
    authors = list(subset.Author.unique())

    # Create Checkbox Widget

    checkbox = CheckboxGroup(labels=authors, active=[])
    
    
show(tab3)

# Intra-PM Tab

In [7]:
'''
Intra-Conversation Tab looks at data between different PM conversations. essentially comparing me to the recipient
for every PM conversation. 

Interesting Column's that I will compare: 'Sent Messages' 'Sent Words' and 'WPM'
Will follow the exact same format Intra-

'''
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, IndexFilter, HoverTool
from bokeh.models.widgets import RadioButtonGroup, CheckboxGroup
from bokeh.layouts import column, row
from bokeh.palettes import turbo


# Import the data
from getData import df



pm_df = df.drop(df[~df.Conversation.isin(list(df.Author.unique()))].index)

metric_selection = ['Sent Messages', 'Sent Words','WPM']
conversations = list(group_df.Conversation.unique())
authors = []

height = 300
width = 1000


def tab1(doc):

    #Create Figures

    top = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    bot = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    #leaderboard = []

    #Create Widgets

    metric_radio = RadioButtonGroup(labels= metric_selection,
                                   active=1)
    conversation_select = Select(title='Conversation',
                                value =conversations[0],
                                options=conversations)


    subset = pm_df[pm_df.Conversation == conversations[0]]
    authors = list(subset.Author.unique())

    # Create Checkbox Widget

    checkbox = CheckboxGroup(labels=authors, active=[])
    
    global old_conversation
    old_conversation = [conversations[0]]
    


    def generateR(subset):
        r_group = {}
        colors = turbo(len(subset.Author.unique()))
        authors = list(subset.Author.unique())
        

        for i, key in enumerate(authors):


            
            src = ColumnDataSource(subset[subset.Author == key])

            r_group[key] = top.line(name = key,x='Datetime',y=metric_radio.labels[metric_radio.active],
                                     source=src, visible = False, legend_label=key, line_width=2,
                                     color=colors[i])
            
            

        return r_group
    
    
    # initialize render_group dictionary
    r_dict ={conversation_select.value : generateR(subset)}
    top.legend.location = 'top_left'
    legend_group = {old_conversation[0] : top.legend.items}
    
    def displayMetric(attr, old, new):
        newval = metric_radio.labels[new]
        for group in r_dict:
            for key in r_dict[group]:
                r_dict[group][key].glyph.y = newval
                

    def displayConversation(attr, old, new):
        old_conversation[0] = old
        checkbox.active = []
        subset = group_df[group_df.Conversation == new]
        authors = list(subset.Author.unique())
        
        top.legend.items = []
        if new not in r_dict:
            r_dict[new] = generateR(subset)
            legend_group[new] = top.legend.items
        else:
            top.legend.items= legend_group[new]
            print()
        old_conversation[0] = new
        checkbox.labels = authors

    def displayAuthor(attr, old, new):
        for i, key in enumerate(checkbox.labels):
            r_dict[old_conversation[0]][key].visible = i in checkbox.active

    conversation_select.on_change('value', displayConversation)
    checkbox.on_change('active', displayAuthor)
    metric_radio.on_change('active', displayMetric)
    layout = column(metric_radio, 
                    row(column(conversation_select, checkbox), top))
    doc.add_root(layout)
    
show(tab1)
    


NameError: name 'group_df' is not defined

# Messing with Moving averages

In [1]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, IndexFilter, HoverTool
from bokeh.models.widgets import RadioButtonGroup, CheckboxGroup
from bokeh.layouts import column, row
from bokeh.palettes import turbo


category = 'Sent Words'
fig1 = figure(x_axis_type = 'datetime', title = 'moving average')
fig2 = figure(x_axis_type = 'datetime', title = 'interpolate')

pm_df = pd.DataFrame()
pm_df = df.copy()

vals = pm_df.loc[pm_df.Author == 'Poop']
vals = vals[['Datetime', category]]
vals.set_index('Datetime',inplace=True)

temp = vals.copy()

vals= vals.resample('7D').first()


vals['difference'] = vals[~vals[category].isnull()].diff()
category='difference'

#vals.interpolate(inplace=True)
vals.fillna(value=0,inplace=True)
#fig1.line(vals.index, vals[category], alpha=0.5, color='navy')
#fig2.line(vals.index, vals[category], alpha=0.5, color='navy')
print(sum(vals[category])/len(vals[category]))

#vals = vals.rolling(window=3).mean()

fig1.line(vals.index, vals[category], alpha=0.4, color='green')
fig2.circle(vals.index, vals[category], alpha=0.1, color='navy',size=2)

vals.fillna(value=0,inplace=True)
print(sum(vals[category])/len(vals[category]))


vals = vals.rolling(window=4).mean()
fig1.line(vals.index, vals[category], alpha=0.7, color='purple')
fig2.line(vals.index, vals[category], alpha=0.3, color='purple')

#vals = temp
vals = temp
vals= vals.resample('7D').first()

vals['difference'] = vals[~vals['Sent Words'].isnull()].diff()


vals.fillna(value=0,inplace=True)

vals = vals.rolling(window=4).mean()




vals.fillna(value=0,inplace=True)


#vals.interpolate(model= 'cubic', order=3,inplace=True)

from scipy.signal import savgol_filter
vals.fillna(value=0,inplace=True)

vals[category] = savgol_filter(vals[category], 11, 3)
fig2.circle(vals.index, vals[category], alpha=0.7, color='green')





vals.fillna(value=0,inplace=True)
fig1.line(vals.index, vals['difference'], alpha=0.3, color ='orange')
fig2.circle(vals.index, vals[category], alpha=0.3, color ='orange', size=3)
print(sum(vals['difference'])/len(vals['difference']))


print(sum(vals[category])/len(vals[category]))

show(row(fig1, fig2))

NameError: name 'pd' is not defined

In [26]:
vals.head()

,Sent Words,difference
Datetime,,
2019-01-23,0.0,2.622378
2019-01-30,0.0,-2.587413
2019-02-06,0.0,-2.599068
2019-02-13,3.0,1.072261
2019-02-20,8.0,6.911422


In [21]:
new_df.head()

,Author,Conversation,Sent Messages,Sent Words,WPM,Total Messages,Total Words
Datetime,,,,,,,
2018-09-28,Miles Keating,Neighbor,1.0,8.0,8.0,1.0,8.0
2018-09-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-09-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-10-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
new_df.Author

Datetime
2018-09-28    Miles Keating
2018-09-29              NaN
2018-09-30              NaN
2018-10-01              NaN
2018-10-02              NaN
                  ...      
2020-08-22    Miles Keating
2020-08-23              NaN
2020-08-24    Miles Keating
2020-08-25            Boner
2020-08-26            Boner
Freq: D, Name: Author, Length: 699, dtype: object

In [22]:
new_df.iloc[:,0]

Datetime
2018-09-28    Miles Keating
2018-09-29              NaN
2018-09-30              NaN
2018-10-01              NaN
2018-10-02              NaN
                  ...      
2020-08-22    Miles Keating
2020-08-23              NaN
2020-08-24    Miles Keating
2020-08-25            Boner
2020-08-26            Boner
Freq: D, Name: Author, Length: 699, dtype: object

# Figure Bot Experiments

In [25]:
new_df = pm_df.copy()
reset_data = {x:[] for x in pm_df.columns.unique()}

new_df.set_index('Datetime',inplace=True)

temp=new_df.copy()

new_df=new_df.resample('D').first()
new_df['difference'] = new_df[~new_df['Author'].isnull()].diff()


all_authors = new_df.Author.unique()
src = {}
for author in all_authors:
    src[author] = ColumnDataSource(reset_data)
    
    
    
    
new_df.head()

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [11]:
reset_data

{'Datetime': [],
 'Author': [],
 'Conversation': [],
 'Sent Messages': [],
 'Sent Words': [],
 'WPM': [],
 'Total Messages': [],
 'Total Words': []}

In [ ]:
debug = figure()
data={'x':[1,2,3,4],'y':[4,8,2,4],'tag':['this','that','this','this']}
src = ColumnDataSource(data)
srcview = CDSView(source=src, filters=[(GroupFilter(column_name='tag',group='this'))])
newbug = debug.circle(source=src,view=srcview,x='x',y='y',color='navy')
show(debug)

# Different callbacks operating on a single graph

In [6]:
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, curdoc
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, IndexFilter, HoverTool
from bokeh.models.widgets import RadioButtonGroup, CheckboxGroup
from bokeh.layouts import column, row
from bokeh.palettes import turbo


# Import the data
from getData import df


output_notebook()

def myapp(doc):

    # Preliminary filter for pm data only
    authors = list(df.Author.unique())
    authors.remove('Miles Keating')
    authors = ['Poop', 'Neighbor']
    pm_df = df[df.Conversation.isin(authors)]
    columns = list(pm_df[['Datetime', 'Author', 'Received Words' , 'Received Messages', 'SMA Sent vs Received Words']].columns)
    
    
    subset = pm_df[columns]
    subset.set_index('Datetime',inplace=True)
    subset= subset.resample('D').first()
    # Create a ColumnDataSource
    pm_cds = ColumnDataSource(subset)
    selection = columns
    
    # Create a RadioButtonGroup and CheckboxGroup
    radio = RadioButtonGroup(labels=selection[2:], active=2) #Datetime and Author columns are not viable selections
    checkbox = CheckboxGroup(labels=authors, active=[0,1,2])
    
    # Create a figure
    colors = turbo(len(authors))
    pm_views = {}
    
    r_dict = {} # preparing a list of renders to change on callback
    p = figure(x_axis_type='datetime',
                                   plot_height=300,
                                   x_axis_label='Date',
                                   y_axis_label= selection[0])
    
    for i, key in enumerate(authors):
        # Create view for each team
        pm_views[key] = CDSView(source=pm_cds,
                                     filters=[GroupFilter(column_name='Author',
                                                          group=key)])
        

        # render each plot
        r_dict[key] = p.step(name = 'poop',x='Datetime',y=radio.labels[radio.active],
                             source=pm_cds, view=pm_views[key], visible = True,
                             color=colors[i])
    p.add_tools(HoverTool(tooltips=[('Author', '@Author')]))

    
    def testfun(attr, old, new):
        change = list(set(old) - set(new) or set(new) - set(old))[0] # Finds the index that was checked
        for i, key in enumerate(checkbox.labels):
            r_dict[key].visible = i in checkbox.active
        
    def changeCol(attr, old, new):
        newval = radio.labels[radio.active]
        for key in authors:
            r_dict[key].glyph.y=newval
    
    radio.on_change('active',changeCol)
    checkbox.on_change('active', testfun)

    doc.add_root(column(radio, row(checkbox, p)))
    doc.title = 'test'
    
    
show(myapp)

Loading BokehJS ...

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=1806&bokeh-absolute-url=http://localhost:44757&resources=none (127.0.0.1)
HTTPServerRequest(protocol='http', host='localhost:44757', method='GET', uri='/autoload.js?bokeh-autoload-element=1806&bokeh-absolute-url=http://localhost:44757&resources=none', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/home/miles/pydir/envy/lib/python3.8/site-packages/tornado/web.py", line 1703, in _execute
    result = await result
  File "/home/miles/pydir/envy/lib/python3.8/site-packages/bokeh/server/views/autoload_js_handler.py", line 60, in get
    session = await self.get_session()
  File "/home/miles/pydir/envy/lib/python3.8/site-packages/bokeh/server/views/session_handler.py", line 120, in get_session
    session = await self.application_context.create_session_if_needed(session_id, self.request, token)
  File "/home/miles/pydir/envy/lib/python3.8/site-packages/bokeh/server/con

In [ ]:
group_df.tail()

# Intra- Group Chats


In [ ]:
'''
This tab will focus on conversations of more than 2 people. All data will be a comparison between all authors
in the conversation.

Ideas
#Should I combine both graphs and overlay the data?
#Should I use range tool?
#


Widget Locations:
Global Metric Option Radio: Top Center
Global Author Select Checkbox Group: Very Left
Fig 1 SMA Radio: above fig1

(top left) Figure 1, will come with moving average options {Hourly, Daily, Weekly, Monthly}
checbkox group to select which authors in the chat to show

(bottom left) Figure 2, will display raw data through tiny circles

(rightFigure 3, will display a chart sorting authors by various metrics

Analytic metrics:

All authors sent messages (by author)
All authors sent Words
All authors cumulative Words per Messages
Emoji usage
Possible analyze media sent
'''

from getData import df
from bokeh.io import show, output_notebook, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, Legend, LegendItem
from bokeh.models.widgets import RadioButtonGroup, Select, CheckboxGroup
from bokeh.layouts import column, row
from bokeh.palettes import turbo

output_notebook()

#Organize data


group_df = df.drop(df[df.Conversation.isin(list(df.Author.unique()))].index)

metric_selection = ['Sent Messages', 'Sent Words','WPM']
conversations = list(group_df.Conversation.unique())
authors = []

height = 300
width = 1000


def tab1(doc):

    #Create Figures

    top = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    bot = figure(x_axis_type='datetime', frame_height=height, frame_width=width)
    #leaderboard = []

    #Create Widgets

    metric_radio = RadioButtonGroup(labels= metric_selection,
                                   active=1)
    conversation_select = Select(title='Conversation',
                                value =conversations[0],
                                options=conversations)


    subset = group_df[group_df.Conversation == conversations[0]]
    authors = list(subset.Author.unique())

    # Create Checkbox Widget

    checkbox = CheckboxGroup(labels=authors, active=[])
    
    global old_conversation
    old_conversation = [conversations[0]]
    


    def generateR(subset):
        r_group = {}
        colors = turbo(len(subset.Author.unique()))
        authors = list(subset.Author.unique())
        

        for i, key in enumerate(authors):


            
            src = ColumnDataSource(subset[subset.Author == key])

            r_group[key] = top.line(name = key,x='Datetime',y=metric_radio.labels[metric_radio.active],
                                     source=src, visible = False, legend_label=key, line_width=2,
                                     color=colors[i])
            
            

        return r_group
    
    
    # initialize render_group dictionary
    r_dict ={conversation_select.value : generateR(subset)}
    top.legend.location = 'top_left'
    legend_group = {old_conversation[0] : top.legend.items}
    
    def displayMetric(attr, old, new):
        newval = metric_radio.labels[new]
        for group in r_dict:
            for key in r_dict[group]:
                r_dict[group][key].glyph.y = newval
                

    def displayConversation(attr, old, new):
        old_conversation[0] = old
        checkbox.active = []
        subset = group_df[group_df.Conversation == new]
        authors = list(subset.Author.unique())
        
        top.legend.items = []
        if new not in r_dict:
            r_dict[new] = generateR(subset)
            legend_group[new] = top.legend.items
        else:
            top.legend.items= legend_group[new]
            print()
        old_conversation[0] = new
        print(top.min_border_left)
        checkbox.labels = authors

    def displayAuthor(attr, old, new):
        for i, key in enumerate(checkbox.labels):
            r_dict[old_conversation[0]][key].visible = i in checkbox.active

    conversation_select.on_change('value', displayConversation)
    checkbox.on_change('active', displayAuthor)
    metric_radio.on_change('active', displayMetric)
    layout = column(metric_radio, 
                    row(column(conversation_select, checkbox), top))
    doc.add_root(layout)
    
show(tab1)
    
# SOME THOUGHTS

'''
It really doesn't seem like adding invisible glyphs and then makign them either visible or invisible on command
is the best way to do this. However if it is the only possible way (Without using JS callbacks) then the next
steps would be to create a custom legend that sits outsiide of the plot and hopefully doesn't mess up the 
axes when new glyphs are added.

The second challenging part will be reseting the x, y and datetime ranges so that I can evnetually add a 
range_slider like one of the interactive examples.

Thirdly will be adding the bottom plot with the raw data and connecting it to all the other widgets. 

'''

    
    
    

print('done')

In [ ]:
            view = CDSView(source=src,
                           filters=[GroupFilter(column_name='Author',
                                                                  group=key)])
    

In [ ]:
group_df.head(200)

In [ ]:
from bokeh.io import curdoc
from bokeh.models import Select, Column
import pandas as pd

output_notebook()

def debug(doc):
    df = pd.DataFrame()
    df['France'] = ['Paris', 'Lione', 'Marseille']
    df['Italy'] = ['Rome','Milan', 'Rimini']
    df['Spain'] = ['Madrid', 'Barcelona', 'Bilbao']
    df['Country']   = ['France', 'Italy', 'Spain']

    select_country = Select(title="Country",  options=list(df['Country']), value = 'France')
    select_city = Select(title = 'Cities', value = 'Paris', options = list(df['France']))

    def update_layout(attr, old, new):
        country_selected = select_country.value
        select_city.options = list(df[country_selected].values)

    select_country.on_change('value', update_layout)
    select_city.on_change('value', update_layout)
    layout = Column(select_country, select_city)
    doc.add_root(layout)
show(debug)

In [ ]:
subset.head(20)

In [ ]:
conversation_select.value

In [ ]:
group_df.Conversation.unique()

# Plotting multiple Graphs using CDS

In [ ]:
from bokeh.io import show, output_file
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, CDSView, GroupFilter
from bokeh.layouts import column
from bokeh.palettes import turbo

# Import the data
from getData import df


# Preliminary filter for pm data only
authors = list(df.Author.unique())
authors.remove('Miles Keating')
authors = ['Bilo', 'Boner']
pm_df = df[df.Conversation.isin(authors)]
columns = pm_df[['Datetime', 'Author', 'Received Words' , 'Received Messages']].columns

# Create a ColumnDataSource
pm_cds = ColumnDataSource(pm_df)

# Create a view for each Author
pm_views = {}
pm_figs = {}
selection = 'Received Words'
colors = turbo(len(authors))
for i, key in enumerate(authors):
    # Create view for each author
    pm_views[key] = CDSView(source=pm_cds,
                                 filters=[GroupFilter(column_name='Author',
                                                      group=key)])
    # Create and configure the figure
    pm_figs[key] = figure(x_axis_type='datetime',
                               plot_height=300,
                               x_axis_label='Date',
                               y_axis_label= selection)
    # Render the graphs
    pm_figs[key].step('Datetime', selection,
                     source=pm_cds, view=pm_views[key],
                     color=colors[i], legend=key)
    # Move the legend to the upper right corner
    pm_figs[key].legend.location = 'top_right'
    
show(column([pm_figs[key] for key in authors]))
    

In [ ]:
'''from bokeh.io import show, output_file
from bokeh.plotting import figure

from bokeh.models.widgets import RadioButtonGroup, Button
from bokeh.palettes import turbo
from bokeh.layouts import column

output_file('deletethis.html')

hover = HoverTool()


p = figure(plot_width=1500, plot_height=1200, x_axis_type='datetime')

author_list = list(pm_df.Author.unique())
author_list.remove('Miles Keating')
colors = turbo(len(author_list))

for i, author in enumerate(author_list):
    subset = pm_df[pm_df.Author == author]
    r= p.line(subset.Datetime, subset['Sent vs Received Words'], line_width=2, color=colors[i], alpha=0.8, legend_label=author)
    r.visible = False

p.title.text = 'Dumb Graph'

p.legend.location = 'top_left'
p.legend.click_policy='hide'

show(p)'''